In [19]:
import csv
import pandas as pd
from mlxtend.frequent_patterns import association_rules

In [20]:
def procedureApriori(T, minSupport):
    candidateItemset = {}
    # frequentItemset is used to keep a running list of all of the frequently occuring sets of data in the code
    frequentItemset = []
    finalCounts = []
    # Generate K1 sets
    for transaction in T:
        for item in transaction:
            if item in candidateItemset:
                candidateItemset[item] += 1
            else:
                candidateItemset[item] = 1
    # Reference used: https://stackoverflow.com/questions/58324419/how-do-i-remove-all-items-in-a-dictionary-with-a-value-less-than-a-variable
    for key in list(candidateItemset):
        if candidateItemset[key] < minSupport:
            del candidateItemset[key]
    frequentItemset = list(candidateItemset)
    if len(candidateItemset) == 0:
        return list(candidateItemset), finalCounts
    finalCounts += list(candidateItemset.values())
    #print(candidateItemset)
    candidateItemset = {}
    countCandidates = {}
    

    # Generate K2 Sets
    # Reference used: https://stackoverflow.com/questions/63970483/how-to-check-if-an-array-contains-all-the-elements-of-another-array-if-not-out
    # Candidate Generation
    for i in range(0, len(frequentItemset)):
        for j in range(i + 1, len(frequentItemset)):
            k2Set = []
            if frequentItemset[i] < frequentItemset[j]:
                currentKey = frequentItemset[i] + " " + frequentItemset[j]
                k2Set.append(frequentItemset[i])
                k2Set.append(frequentItemset[j])
            else:
                currentKey = frequentItemset[i] + " " + frequentItemset[j]
                k2Set.append(frequentItemset[j])
                k2Set.append(frequentItemset[i])
            candidateItemset[currentKey] = k2Set
            # Transaction Iteration
            for transaction in T:
                if (
                    len(
                        [
                            h
                            for h in candidateItemset[currentKey]
                            if h not in transaction
                        ]
                    )
                    == 0
                ):
                    if currentKey in countCandidates:
                        countCandidates[currentKey] += 1
                    else:
                        countCandidates[currentKey] = 1
    # Candidate Pruning
    for key in list(candidateItemset):
        if key in countCandidates:
            if countCandidates[key] < minSupport:
                del candidateItemset[key]
                del countCandidates[key]
            else:
                frequentItemset.append(key)
        else:
            del candidateItemset[key]

    finalCounts += list(countCandidates.values())



    if len(candidateItemset) == 0:
        return frequentItemset, finalCounts
    k = 3

    while True:
        countCandidates = {}
        newCandidates = {}
        currentSet = list(candidateItemset)
        # Candidate Generation
        for i in range(0, len(candidateItemset)):
            for h in range(i + 1, len(candidateItemset)):
                match = True
                for x in range(0, len(currentSet[i]) - 2):
                    if len(currentSet[i]) != len(currentSet[h]):
                        continue
                    if currentSet[i][x] != currentSet[h][x]:
                        match = False
                if match == True:
                    newElement = (
                        currentSet[i] + " " + currentSet[h][len(currentSet[h]) - 1]
                    )
                    if newElement not in newCandidates:
                        L = candidateItemset[currentSet[i]]
                        L.append(currentSet[h][len(currentSet[h]) - 1])
                        newCandidates[newElement] = L

        # Transaction Iteration
        newCountCandidates = {}
        for key in newCandidates:
            for transaction in T:
                if len([h for h in newCandidates[key] if h not in transaction]) == 0:
                    if key in newCountCandidates:
                        newCountCandidates[key] += 1
                    else:
                        newCountCandidates[key] = 1
        # Candidate Pruning
        for key in list(newCandidates):
            if key in newCountCandidates:
                if newCountCandidates[key] < minSupport:
                    del newCandidates[key]
                    del newCountCandidates[key]
                else:
                    frequentItemset.append(key)
            else:
                del newCandidates[key]

        finalCounts += list(newCountCandidates.values())

        

        # If no candidates after pruning, return
        if len(newCandidates) == 0:
            return frequentItemset, finalCounts
        candidateItemset = newCandidates

        k += 1

In [21]:
import pandas as pd
import numpy as np

data = pd.read_csv("covid.csv")
data.head()

,Tweet,Emojiset
0,RT maykahangtoran: We really made VP Leni Robr...,😭
1,RT DEATHBALL13: Meanwhile 🤡🤡Joey you’re hiding...,🤡🤡
2,RT minimeuu: Já apanhaste COVID!? Me:não 😬 RT ...,😬
3,RT sarinhaavps: Já apanhaste COVID!? Me:não 😬 ...,😬
4,RT fann961: 7-day average daily Covid deaths p...,"🇭🇰,🇺🇸,🇯🇵,🇬🇧,🇨🇦,🇦🇺,🇸🇬"


In [22]:
from emoji import UNICODE_EMOJI

def extractEmojis(text):
    emojis = []

    if type(text) != str:
        return []
    
    for word in text.split(" "):
        if word in UNICODE_EMOJI['en']:
            emojis.append(word)
        else:
            current_emoji = ""

            for c in range(len(word)):
                if word[c] in UNICODE_EMOJI['en']:
                    current_emoji += word[c]

            if current_emoji != "":
                emojis.append(current_emoji)
                current_emoji = ""

    return emojis

# UNICODE_EMOJI['en']

In [23]:
data['extracted_emojis'] = data['Tweet'].apply(extractEmojis)
data.head()

,Tweet,Emojiset,extracted_emojis
0,RT maykahangtoran: We really made VP Leni Robr...,😭,[😭]
1,RT DEATHBALL13: Meanwhile 🤡🤡Joey you’re hiding...,🤡🤡,[🤡🤡]
2,RT minimeuu: Já apanhaste COVID!? Me:não 😬 RT ...,😬,[😬]
3,RT sarinhaavps: Já apanhaste COVID!? Me:não 😬 ...,😬,[😬]
4,RT fann961: 7-day average daily Covid deaths p...,"🇭🇰,🇺🇸,🇯🇵,🇬🇧,🇨🇦,🇦🇺,🇸🇬","[🇭🇰, 🇺🇸, 🇯🇵, 🇬🇧, 🇨🇦, 🇦🇺, 🇸🇬]"


In [24]:
inputList = data['extracted_emojis']
inputList.head(50)

0                              [😭]
1                             [🤡🤡]
2                              [😬]
3                              [😬]
4     [🇭🇰, 🇺🇸, 🇯🇵, 🇬🇧, 🇨🇦, 🇦🇺, 🇸🇬]
5                              [😅]
6                           [🕯, 🕯]
7                           [🤣🤣🤣🤣]
8                              [🗣]
9                          [😡⬇, 😕]
10                             [🤬]
11                             [😭]
12                      [🪙🙏, 🙏, 🪙]
13                             [😭]
14                          [🤣🤣🤣🤣]
15                             [▶]
16                      [🦠, 💉, 🙌🏻]
17                          [🤣🤣🤣🤣]
18                            [☝🏻]
19                         [😡⬇, 😕]
20                        [🤦♀🤦♀🤦♀]
21                          [🤧, 💚]
22                             [💡]
23                         [🚨, 🤬🤮]
24                     [🔬, ➡️, ✍️]
25                             [🤮]
26                          [🌞, 😷]
27                            [👍🏼]
28                  

In [35]:
setSup = 3

In [36]:
inputList

0                                [😭]
1                               [🤡🤡]
2                                [😬]
3                                [😬]
4       [🇭🇰, 🇺🇸, 🇯🇵, 🇬🇧, 🇨🇦, 🇦🇺, 🇸🇬]
                    ...             
996                             [💉💉]
997                           [☑, ☑]
998                              [👑]
999                           [✅, 😷]
1000                              []
Name: extracted_emojis, Length: 1001, dtype: object

In [27]:
x, counts = procedureApriori(inputList, setSup)
print(x)
print(counts)

['😭', '🤡🤡', '😬', '🇺🇸', '🇬🇧', '🇸🇬', '😅', '🤣🤣🤣🤣', '🗣', '😡⬇', '😕', '🤬', '🪙🙏', '🙏', '🪙', '▶', '🦠', '💉', '🙌🏻', '💡', '🚨', '🤬🤮', '🔬', '➡️', '✍️', '😷', '👍🏼', '🖐️', '🤣', '😭😭', '💉💉', '💕', '😃', '🤔', '👇', '👉', '🏥', '🤡', '😍', '✅', '🧵', '🙄', '🗣️', '😷👏', '😩', '👍', '🥴', '😎', '❤', '✈️', '🛫', '😂', '🔸', '™', '😁', '🇩🇴', '😳', '😉', '❌', '🗞️', '🔴', '➡', '🤫', '😈', '😥', '🤦\u200d♂️', '🛑', '🇧🇷', '😡🚩', '😭😤', '👏', '👀', '🛡', '📍', '🐭', '🌎', '🤣🤣🤣', '🥺', '😔', '‼', '🐱💭', '👍🏽', '✔', '🩹', '➕', '👉🏽', '😱', '😊', '🤯', '🧟\u200d♂️', '📢', '🥺🤏🏻', '🧪', '📈➡', '💢', '😭😭😭', '🤣🤣', '🚩🥰❤', '👇🏽', '🕊🚩💉', '😂😂😂😂😂', '🇺🇦', '😆', '⬇', '🐥', '📌', '🧬', '♨', '🔥', '😏', '🟢', '🔹', '🤨', '😅 🤣🤣🤣', '😡⬇ 😕', '🪙🙏 🙏', '🪙🙏 🪙', '🙏 🪙', '🙏 🙄', '▶ 🔸', '🦠 💉', '🦠 🙌🏻', '🦠 🤔', '🦠 👍🏽', '🦠 ✔', '🦠 🩹', '🦠 ➕', '🦠 👉🏽', '💉 🙌🏻', '🚨 🤬🤮', '🔬 ➡️', '🔬 ✍️', '🔬 🧪', '🔬 📈➡', '➡️ ✍️', '😷 ✅', '💕 😃', '🤔 👍🏽', '🤔 ✔', '🤔 🩹', '🤔 ➕', '🤔 👉🏽', '👇 👉', '👇 🏥', '👇 ➡', '👇 🐭', '👉 🏥', '👉 📌', '🏥 🔴', '✈️ 🛫', '™ 😁', '➡ 🐭', '🤫 😈', '🇧🇷 😡🚩', '👍🏽 ✔', '👍🏽 🩹', '👍🏽 ➕', '👍🏽 👉🏽', '✔ 🩹', '✔ ➕', '✔ 👉🏽', '🩹 ➕', '🩹 

In [28]:
frozen_new_x = []
new_x = []
for i in x:
    frozen_new_x.append(frozenset([k for k in i.split(" ")]))
    new_x.append([k for k in i.split(" ")])

In [39]:
frequent_itemsets = pd.DataFrame()
frequent_itemsets["itemsets"] = frozen_new_x
frequent_itemsets["support"] = counts

In [40]:
frequent_itemsets.head()

,itemsets,support
0,(😭),116
1,(🤡🤡),10
2,(😬),12
3,(🇺🇸),9
4,(🇬🇧),3


In [30]:
r = association_rules(frequent_itemsets, support_only=True)

In [31]:
rules = []
for l in range(len(r["antecedents"].values)):
    rules.append([list(r["antecedents"][l])[0], list(r["consequents"][l])[0]])

In [32]:
len(rules)

112

In [33]:
new_rules = []

for i in new_x:
    for rule in rules:
        before = rule[0]
        after = rule[1]
        before_idx = None
        after_idx = None
        for item in i:
            if item == before:
                if before_idx == None:
                    before_idx = i.index(item)
            elif item == after:
                if after_idx == None:
                    after_idx = i.index(item)
        if before_idx != None and after_idx != None:
            if before_idx < after_idx:
                if rule not in new_rules:
                    new_rules.append(rule)
print(new_rules)

[['😅', '🤣🤣🤣'], ['😡⬇', '😕'], ['🪙🙏', '🙏'], ['🪙🙏', '🪙'], ['🙏', '🪙'], ['🙏', '🙄'], ['▶', '🔸'], ['🦠', '💉'], ['🦠', '🙌🏻'], ['🦠', '🤔'], ['🦠', '👍🏽'], ['🦠', '✔'], ['🦠', '🩹'], ['🦠', '➕'], ['🦠', '👉🏽'], ['💉', '🙌🏻'], ['🚨', '🤬🤮'], ['🔬', '➡️'], ['🔬', '✍️'], ['🔬', '🧪'], ['🔬', '📈➡'], ['➡️', '✍️'], ['😷', '✅'], ['💕', '😃'], ['🤔', '👍🏽'], ['🤔', '✔'], ['🤔', '🩹'], ['🤔', '➕'], ['🤔', '👉🏽'], ['👇', '👉'], ['👇', '🏥'], ['👇', '➡'], ['👇', '🐭'], ['👉', '🏥'], ['👉', '📌'], ['🏥', '🔴'], ['✈️', '🛫'], ['™', '😁'], ['➡', '🐭'], ['🤫', '😈'], ['🇧🇷', '😡🚩'], ['👍🏽', '✔'], ['👍🏽', '🩹'], ['👍🏽', '➕'], ['👍🏽', '👉🏽'], ['✔', '🩹'], ['✔', '➕'], ['✔', '👉🏽'], ['🩹', '➕'], ['🩹', '👉🏽'], ['➕', '👉🏽'], ['🧪', '📈➡'], ['🚩🥰❤', '👇🏽'], ['🚩🥰❤', '🕊🚩💉'], ['👇🏽', '🕊🚩💉'], ['♨', '🔥']]


In [34]:
len(new_rules)

56